In [2]:
#importing essential modules
import pandas as pd
import numpy as np
import os,sys
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder,StandardScaler
import tensorflow as tf

In [3]:
#importing local modules
sys.path.append(os.path.abspath(os.path.join('../scripts')))

from clean import Clean
from utils import vocab
from deep_learner import DeepLearn
from modeling import Modeler
from evaluator import CallbackEval

In [4]:
#mapping an object
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'